In [2]:
# Environment

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
# Hyper parameters

n_epochs = 5
batch_size_train = 100
batch_size_test = 1000
learning_rate = 0.001
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enable = False
torch.manual_seed(random_seed)


In [4]:
# Load MNIST dataset in a handy way

train_loader = torch.utils.data.DataLoader(
    dsets.MNIST('/files/', train=True, download=True, 
                transform=transforms.Compose([transforms.ToTensor(),
                                              transforms.Normalize((0.1307,), (0.3081,))
                                             ])),
    batch_size=batch_size_train, shuffle=True)

train_loader = torch.utils.data.Dataloader(
    dsets.MNIST('/files/', train=False, download=True, 
                transform=transforms.Compose([transforms.ToTensor(),
                                              transforms.Normalize((0.1307,), (0.3081,))
                                             ])),
    batch_size=batch_size_test, shuffle=True)

PermissionError: [Errno 13] Permission denied: '/files'